# Building a ReAct Agent from Scratch

This notebook implements a ReAct (Reasoning + Acting) agent using AWS Bedrock.

The agent follows this pattern:
1. **Thought**: Reasons about what to do
2. **Action**: Decides an action to take
3. **Observation**: Receives result from action
4. **Repeat**: Continues until task is complete

## Use Case: Customer Support Ticketing System

Our agent will help manage customer support queries by:
- Looking up customer information
- Checking ticket statuses
- Performing calculations when needed

In [ ]:
# Install required packages
!pip install -q langchain langchain-aws boto3

In [ ]:
import os
import boto3
from langchain_aws import ChatBedrock
from langchain_core.tools import tool
import requests
from datetime import datetime
import json
from google.colab import userdata

# Import Google Colab userdata for secure credential access
from google.colab import userdata

# Configure your AWS credentials using Colab secrets
AWS_ACCESS_KEY_ID = userdata.get('awsid')  # Set this in Colab secrets
AWS_SECRET_ACCESS_KEY = userdata.get('awssecret')  # Set this in Colab secrets
AWS_REGION = "us-east-1"  # Change if needed

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

print("✓ AWS Bedrock client initialized")

# Set up the Bedrock model (using Amazon Nova Lite for cost-effectiveness)
llm = ChatBedrock(
    client=bedrock_runtime,
    model_id="amazon.nova-lite-v1:0",
    model_kwargs={
        "temperature": 0,  # Low temperature for consistent tool calling
        "max_tokens": 4096
    }
)

In [ ]:
# Import additional libraries
import re
from typing import List, Dict, Any
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
# Test the AWS Bedrock LLM with a simple message
def call_llm(messages: List[Dict[str, Any]]) -> str:
    """
    Call AWS Bedrock with messages.
    """
    # Convert to LangChain message format
    lc_messages = []
    for msg in messages:
        if msg["role"] == "user":
            lc_messages.append(HumanMessage(content=msg["content"]))
        elif msg["role"] == "assistant":
            lc_messages.append(AIMessage(content=msg["content"]))

    # Call the model
    response = llm.invoke(lc_messages)
    return response.content

# Test it
test_response = call_llm([{"role": "user", "content": "Hello, world!"}])
print(test_response)

## Define the Agent Class

The agent maintains a conversation history and executes the ReAct loop.

In [ ]:
class Agent:
    def __init__(self, system: str = ""):
        """
        Initialize the agent with an optional system message.
        """
        self.system = system
        self.messages: List[Dict[str, str]] = []
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message: str) -> str:
        """
        Process a message and return the agent's response.
        """
        # Add user message to conversation history
        self.messages.append({"role": "user", "content": message})

        # Get response from the model
        result = self.execute()

        # Add assistant message to conversation history
        self.messages.append({"role": "assistant", "content": result})

        return result

    def execute(self) -> str:
        """
        Execute the model call with current messages.
        """
        # Convert to LangChain messages
        lc_messages = []
        for msg in self.messages:
            if msg["role"] == "system":
                from langchain_core.messages import SystemMessage
                lc_messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                lc_messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                lc_messages.append(AIMessage(content=msg["content"]))

        response = llm.invoke(lc_messages)
        return response.content

print("✓ Agent class defined")

## Create the ReAct System Prompt

This prompt instructs the model how to use the Thought-Action-Observation loop for customer support tasks.

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 9 * 2 / 4
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_customer_info:
e.g. get_customer_info: CUST12345
Returns customer information including name, plan type, and account status when given a customer ID

get_ticket_status:
e.g. get_ticket_status: TKT98765
Returns the current status and details of a support ticket when given a ticket ID

Example session:

Question: What is the status of ticket TKT98765 for customer CUST12345?
Thought: I need to first get the customer information, then check the ticket status
Action: get_customer_info: CUST12345
PAUSE

You will be called again with this:

Observation: Customer: Sarah Johnson, Plan: Premium, Status: Active

You then continue:

Thought: Now I have the customer info, let me check the ticket status
Action: get_ticket_status: TKT98765
PAUSE

You will be called again with this:

Observation: Ticket TKT98765: Status: In Progress, Issue: Billing inquiry, Priority: Medium, Assigned to: Agent Mike

You then output:

Answer: Ticket TKT98765 belongs to Sarah Johnson (Premium plan, Active status). The ticket is currently In Progress, regarding a billing inquiry with Medium priority, and is assigned to Agent Mike.
""".strip()

print("✓ System prompt defined")
print("\nPrompt preview:")
print(prompt[:300] + "...")

## Define Available Tools/Actions

These are the functions the agent can call during execution.

In [ ]:
def calculate(operation: str) -> float:
    """
    Perform a mathematical calculation.
    """
    return eval(operation)

def get_customer_info(customer_id: str) -> str:
    """
    Return customer information based on customer ID.
    This is a mock implementation with hardcoded values.
    """
    customer_id = customer_id.strip().upper()

    customers = {
        'CUST12345': 'Customer: Sarah Johnson, Plan: Premium, Status: Active',
        'CUST67890': 'Customer: John Smith, Plan: Basic, Status: Active',
        'CUST11111': 'Customer: Emily Davis, Plan: Enterprise, Status: Active'
    }

    return customers.get(customer_id, f"Customer not found: {customer_id}")

def get_ticket_status(ticket_id: str) -> str:
    """
    Return ticket status and details based on ticket ID.
    This is a mock implementation with hardcoded values.
    """
    ticket_id = ticket_id.strip().upper()

    tickets = {
        'TKT98765': 'Ticket TKT98765: Status: In Progress, Issue: Billing inquiry, Priority: Medium, Assigned to: Agent Mike',
        'TKT11111': 'Ticket TKT11111: Status: Resolved, Issue: Password reset, Priority: Low, Assigned to: Agent Lisa',
        'TKT22222': 'Ticket TKT22222: Status: Open, Issue: Feature request, Priority: High, Assigned to: Agent Tom'
    }

    return tickets.get(ticket_id, f"Ticket not found: {ticket_id}")

# Dictionary mapping action names to functions
known_actions = {
    "calculate": calculate,
    "get_customer_info": get_customer_info,
    "get_ticket_status": get_ticket_status
}

print("✓ Tools defined:")
for action in known_actions.keys():
    print(f"  - {action}")

## Manual Agent Execution (Step-by-step)

Let's first run the agent manually to understand each step.

In [ ]:
# Initialize agent with the ReAct prompt
agent = Agent(system=prompt)

# Step 1: Ask the question
result = agent("What is the status of ticket TKT98765?")
print("Step 1 - Agent's first response:")
print(result)
print("\n" + "="*50 + "\n")

In [ ]:
# Step 2: Execute the action manually
# Parse the action from the response
action_match = re.search(r'Action: ([a-z_]+): (.+)', result, re.IGNORECASE)
if action_match:
    action_name = action_match.group(1).strip()
    action_input = action_match.group(2).strip()

    print(f"Executing: {action_name}({action_input})")

    # Call the function
    observation = known_actions[action_name](action_input)
    print(f"Observation: {observation}")

    # Format the observation for the agent
    next_prompt = f"Observation: {observation}"
    print("\n" + "="*50 + "\n")

In [ ]:
# Step 3: Pass observation back to agent
result = agent(next_prompt)
print("Step 3 - Agent's final response:")
print(result)
print("\n" + "="*50 + "\n")

In [ ]:
# View the complete message history
print("Complete conversation history:")
for i, msg in enumerate(agent.messages):
    print(f"\n{i+1}. {msg['role'].upper()}:")
    print(msg['content'][:200] + ("..." if len(msg['content']) > 200 else ""))

## Complex Example (Manual)

Let's try a more complex question that requires multiple steps.

In [ ]:
# Reinitialize the agent
agent = Agent(system=prompt)

# Ask a complex question
question = "Get me the details for customer CUST12345 and their ticket TKT98765"
result = agent(question)
print("Question:", question)
print("\nAgent response:")
print(result)
print("\n" + "="*50)

## Automated Agent Loop

Now let's automate the entire ReAct loop.

In [ ]:
# Regex pattern to find actions in the response
action_re = re.compile(r'^Action: (\w+): (.*)$', re.MULTILINE)

def query(question: str, max_turns: int = 5) -> str:
    """
    Run the agent in an automated loop until it provides an answer.
    """
    i = 0
    agent = Agent(system=prompt)
    next_prompt = question

    while i < max_turns:
        i += 1
        print(f"\n{'='*60}")
        print(f"Turn {i}")
        print(f"{'='*60}")

        # Get agent's response
        result = agent(next_prompt)
        print(result)

        # Parse for actions
        actions = [
            (action_match.group(1), action_match.group(2))
            for action_match in action_re.finditer(result)
        ]

        # If there are actions to take
        if actions:
            action, action_input = actions[0]

            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")

            print(f"\n→ Executing: {action}({action_input})")
            observation = known_actions[action](action_input)
            print(f"→ Observation: {observation}")

            next_prompt = f"Observation: {observation}"
        else:
            # No actions found, agent is done
            print("\n✓ Agent finished!")
            return result

    print("\n⚠ Max turns reached")
    return result

print("✓ Query function defined")

## Run the Automated Agent

Let's test the agent with various customer support scenarios.

In [ ]:
# Simple question - check ticket status
result = query("What is the status of ticket TKT98765?")

In [ ]:
# Complex question requiring multiple steps
result = query("Get me the details for customer CUST12345 and their ticket TKT98765")

In [ ]:
# Question about customer information
result = query("Who is customer CUST67890 and what plan are they on?")

In [ ]:
# Multiple ticket statuses
result = query("What are the statuses of tickets TKT11111 and TKT22222?")

In [ ]:
# Question with calculation
result = query("If we have 15 high priority tickets and 23 medium priority tickets, what's the total number of tickets?")

In [ ]:
# Complex scenario with customer info and calculation
result = query("Customer CUST11111 has 3 open tickets and customer CUST67890 has 5 open tickets. What's their combined ticket count?")

## Summary

In this notebook, we built a ReAct agent from scratch that:

1. **Thinks** about customer support problems using natural language reasoning
2. **Acts** by calling available tools/functions:
   - `get_customer_info`: Retrieves customer details
   - `get_ticket_status`: Checks support ticket information
   - `calculate`: Performs mathematical operations
3. **Observes** the results of those actions
4. **Repeats** until it can provide a final answer

### Key Components:

- **Agent Class**: Manages conversation history and model calls
- **System Prompt**: Instructs the model on the ReAct pattern
- **Tools**: Functions the agent can call (customer info, ticket status, calculations)
- **Query Loop**: Automates the Thought-Action-Observation cycle

### Real-World Applications:

This pattern can be extended for:
- **Customer Support**: Automated ticket triage and information lookup
- **E-commerce**: Order tracking, inventory management
- **Data Analysis**: Query databases, perform calculations, generate reports
- **DevOps**: System monitoring, log analysis, incident response

The ReAct pattern is powerful because it combines:
- Natural language understanding (LLM)
- Structured tool execution (Python functions)
- Iterative problem-solving (loop until complete)

This foundation can be extended with:
- Database connections
- API integrations
- More sophisticated tools
- Error handling and retries
- Conversation memory and context

In [ ]:
from IPython.display import Image, display

# Display the image from the URL
display(Image(url='https://raw.githubusercontent.com/enoch-sit/project-1-ipynb/refs/heads/main/images/nativeAgent.svg'))

# Group discussion

**Will it works for the EDUHK endpoint?**

- Why?